In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
dataset = pd.read_csv("dataset.txt", delimiter="|", encoding="ISO-8859-1", index_col="AutoID")
dataset.head()

,Date,Year,Month,MediaType,FullText
AutoID,,,,,
1,8/26/2015,2015,8,twitter,3 ways the internet of things will change Bank...
2,8/5/2015,2015,8,twitter,BankB BankB Name downgrades apple stock to neu...
3,8/12/2015,2015,8,twitter,BankB returns to profit on INTERNET/! board2? ...
4,8/5/2015,2015,8,twitter,BankB tells advisers to exit paulson hedge fun...
5,8/12/2015,2015,8,twitter,BankC may plead guilty over foreign exchange p...


In [3]:
def filter_terms(in_str, sub_list=None):
    if sub_list is None:
        return in_str

    for pattern in sub_list:
        in_str = re.sub("\\b" + pattern + "\\b", '', in_str)

    return in_str

In [4]:
# customer service
s = re.compile("customer service", re.IGNORECASE)
mask = (dataset["MediaType"] == "twitter") &\
       (dataset["FullText"].apply(lambda x: 1 if s.search(x) else 0))
#        (dataset["FullText"].apply(lambda x: len(x)) > 1000)


print("Comments that meet criteria: %s" % len(dataset[mask]))

# sub_list = ["ADDRESS", "INTERNET", "Name", 
#             "twit_hndl", "PHONE", "twit_hndl_BankA", 
#             "twit_hndl_BankB", "twit_hndl_BankC", "twit_hndl_BankD"
#             "BankA", "BankB", "BankC", "BankD"]
# sub_list += nltk.corpus.stopwords.words('english')

for comment in dataset[mask]["FullText"]:
    print(comment)
#     print(filter_terms(comment, sub_list))
    print()

Comments that meet criteria: 1175
. twit_hndl_BankB is the worst bank ever! horrible customer service service and you guys dont help. worthless.

. twit_hndl_BankB your customer service sucks sucks. thanks to the 7 agents in the last 2 days days that couldnt help me, 1 that tried, and 1 liar.

. twit_hndl_BankB, holding on your customer service line line is tedious, but would be less tedious if twit_hndl was the voice on the line.

. twit_hndl_BankA twit_hndl_BankA amazing customer service service as usual- thanks for all the help

. twit_hndl_BankA i did thank you very much your customer service and claims dept. were very helpful.

. twit_hndl_BankC definitely has the worst customer service ever. cant wait to cancel this card and stick with twit_hndl.

. twit_hndl_BankC supervisor puts me on hold for 45 minutes minutes, then hangs up the phone. quality customer service service you have there.

. twit_hndl_BankC thanks so much for the call!! great customer service

. twit_hndl_BankC Na

## Key Phrases

In [16]:
from nltk.corpus import wordnet as wn, brown

def get_sents(collection):
    sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(collection) 
    return [nltk.word_tokenize(word) for word in raw_sents]

def build_backoff_tagger(train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    return t2

def get_words_with_tag(collection_tagged, tag):
    collection_words = [w[0] for s in collection_tagged for w in s if w[1].startswith(tag) and
                        w[0].lower() not in stopwords and len(w[0]) > 3]
    return collection_words

# vp_chunker = nltk.RegexpParser('VP: {(<RB|BED.*|MD|HV|VB.*|IN>+<V.*>) | (<V.*><RB|V.*>*)}')
# noun_chunker = nltk.RegexpParser('NP: {(<JJ.*|N.*>+<N.*|CD>)}')

def get_phrase_freq(chunker, collection_tagged, label, amount):
    phrases = [] 
    for sent in collection_tagged:   
        tree = chunker.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == label: 
                phrases.append(str(subtree))
    np_freqdist = nltk.FreqDist(phrases)
    return np_freqdist.most_common(amount)

def key_phrases(in_list):
    noun_chunker = nltk.RegexpParser('NP: {(<NN.*|N.*|PP.*>+<N.*|CD>)}')
    noun_phrases = noun_chunker.parse(tagged)
    return noun_phrases

In [17]:
brown_tagged_sents = brown.tagged_sents()
ngram_tagger = build_backoff_tagger(brown_tagged_sents)

tagged = [ngram_tagger.tag_sents(get_sents(comment)) for comment in dataset[mask]["FullText"]]
print(key_phrases(tagged[0]))
print()

ValueError: chunk structures must contain tagged tokens or trees

## Sample Dataset

In [145]:
rows = np.random.choice(dataset.index.values, 10000)
sample = dataset.ix[rows]

## DictVectorizer

In [167]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
cols = ["has_banka", "has_bankb", "has_bankc", "has_bankd", "has_banka_handle", "has_bankb_handle",
        "has_bankc_handle", "has_bankd_handle", "has_address", "has_link", "has_phone", "has_name"]
vec = vec.fit_transform(sample[cols].transpose().to_dict().values())

/Users/vijayv/anaconda/lib/python3.4/site-packages/pandas/core/frame.py:711: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  "columns will be omitted.", UserWarning)


In [170]:
from sklearn.cluster import KMeans
num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(vec)

clusters = km.labels_.tolist()

CPU times: user 2.41 s, sys: 16 ms, total: 2.42 s
Wall time: 2.59 s


In [ ]:
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :20]:
        print(" %s" % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end=",")
    print()
    print()
    
print()
print()

In [111]:
from sklearn.externals import joblib

joblib.dump(km, 'cluster_algo1.pkl')

# km = joblib.load('cluster_algo1.pkl')

['cluster_algo1.pkl', 'cluster_algo1.pkl_01.npy', 'cluster_algo1.pkl_02.npy']